# NYC Taxi Non-geographic
Written by James A. Bednar<br>
Created: January 29, 2016<br>
Last updated: August 4, 2021

## Plotting non-geographic data

Most of the datashader examples use geographic data, because it is so easily interpreted, but datashading will help exploration of any data dimensions.  Here let's start by plotting `trip_distance` versus `fare_amount` for the 12-million-point NYC taxi dataset from nyc_taxi.ipynb. 

In [ ]:
import numpy as np
import holoviews as hv
import holoviews.operation.datashader as hd
import datashader as ds
from holoviews import opts
hv.extension('bokeh')

In [ ]:
opts.defaults(
    opts.Scatter(width=800, height=500, color='blue'),
    opts.RGB(width=800, height=500),
    opts.Curve(width=800))

### Load NYC Taxi data 

These data have been transformed from the original database to a [parquet file](https://parquet.apache.org/documentation/latest/). It should take about 5 seconds to load (compared to 10-20 seconds when stored in the inefficient CSV file format...).

In [ ]:
import dask.dataframe as dd

usecols = ['trip_distance','fare_amount','tip_amount','passenger_count']
%time df = dd.read_parquet('data/nyc_taxi_wide.parq')[usecols].persist()
df.tail()

### 1000 points reveals the expected linear relationship

In [ ]:
samples = df.sample(frac=1e-4)
scatter = hv.Scatter(samples, 'trip_distance', 'fare_amount')
labelled = scatter.redim.label(trip_distance="Distance, miles", fare_amount="Fare, $")
labelled.redim.range(trip_distance=(0, 20), fare_amount=(0,40)).opts(size=5)

### 10,000 points show more detailed, systematic patterns in fares and times
  
Perhaps there are different metering options, along with granularity in how times and fares are counted; in any case, the times and fares do not uniformly populate any region of this space:

In [ ]:
samples = df.sample(frac=1e-3)
scatter = hv.Scatter(samples, 'trip_distance', 'fare_amount')
labelled = scatter.redim.label(trip_distance="Distance, miles", fare_amount="Fare, $")
labelled.redim.range(trip_distance=(0, 20), fare_amount=(0,40)).opts(alpha=0.05, size=1)

### Datashader reveals additional detail, especially when zooming in

You can now see that there are a lot of points below the linear boundary, representing long trips for very little cost (presumably GPS errors?).

In [ ]:
scatter = hv.Scatter(df, 'trip_distance', 'fare_amount')
ranged = scatter.redim.range(trip_distance=(0, 20), fare_amount=(0,40))
shaded = hd.spread(hd.datashade(ranged))
labelled = shaded.redim.label(trip_distance="Distance, miles", fare_amount="Fare, $")
labelled

Here we're using the default histogram-equalized color mapping function to reveal density differences across this space.  If we used a linear mapping (same code as above wirh `normalization='linear'`), we can mainly see that there are a lot of values near the origin, but all the rest are colored the same minimum (defaulting to light blue) color:

In [ ]:
shaded = hd.spread(hd.datashade(ranged, normalization='linear'))
labelled = shaded.redim.label(trip_distance="Distance, miles", fare_amount="Fare, $")
labelled

Fares are discretized to the nearest 50 cents, making patterns less visible, but there is both an upward trend in tips as fares increase (as expected), but also a large number of tips higher than the fare itself, which is surprising:

In [ ]:
scatter = hv.Scatter(df, 'tip_amount', 'fare_amount')
ranged = scatter.redim.range(tip_amount=(0,40), fare_amount=(0,20))
shaded = hd.spread(hd.datashade(ranged))
labelled = shaded.redim.label(tip_amount="Tip, $", fare_amount="Fare, $")
labelled

Interestingly, tips go down when the number of passengers is greater than 1:

In [ ]:
scatter = hv.Scatter(df, 'passenger_count', 'tip_amount')
ranged = scatter.redim.range(tip_amount=(0,60), passenger_count=(-0.5,6.5))
shaded = hd.spread(hd.datashade(ranged, x_sampling=0.15))
labelled = shaded.redim.label(passenger_count="Passengers", tip_amount="Tip, $")
labelled

Here we've reduced the resolution along the x axis so that instead of getting isolated points for this inherently discrete data, you can see more-visible horizontal line segments.

The above plots use the [HoloViews](http://holoviews.org) library, which builds Bokeh and Matplotlib plots from high-level specifications.  For instance, Datashader currently only provides 2D aggregates, but you can easily make a zoomable one-dimensional histogram using HoloViews to dynamically collapse across a second dimension:

In [ ]:
dataset = hv.Points(df, kdims=['fare_amount', 'trip_distance'], vdims=[]).select(fare_amount=(0,60))
agg = hd.rasterize(dataset, aggregator=ds.count(), streams=[hv.streams.RangeX()], x_sampling=0.5, width=500, height=2)
agg.apply.reduce(trip_distance=np.sum)

Here datashader is aggregating over both fare_amount and trip_distance, but trip_distance was specified to have only a height of 2, because it will be further collapsed to create the histogram being displayed.  You can now use the wheel zoom tool when hovering over the x axis, and the plot will zoom in or out, dynamically resampling at the given location to make a new histogram (as long as there is a live Python server running). 

In this particular plot, there is a very wide range of fare amounts, with an implausibly high maximum fare of over 4000 dollars, but you can easily zoom in to the bulk of the data to show that nearly all fares are between 4 and 20 dollars, following something like a gamma distribution, and they are discretized to the nearest 50 cents in this dataset.